### Visualisation

In [2]:
# !pip install pandas openpyxl pillow

In [6]:
import pandas as pd
import numpy as np
import json
import os
from PIL import Image

from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage

# base_folder = r"D:\Projects\OMR\new_abhigyan\Restructure"

# --- Configuration ---
EXCEL_FILE_PATH = r"D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out.xlsx" 
CSV_FILE_PATH = r"D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\options_BATCH05\ed_results_human.csv"     
VERIFICATION_FILE_PATH = r"D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\options_BATCH05\verification.csv"
JSON_FILE_PATH = r"D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\BATCH05.json"

ORIGINAL_IMAGES_FOLDER_PATH = r"D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Input\BATCH05"

OUTPUT_EXCEL_FILE = 'comparison_results.xlsx'
CROPPED_IMAGES_DIR = 'cropped_question_images' # Folder to save cropped images

print("--- Loading Data ---")

# Load Excel
try:
    excel_df = pd.read_excel(EXCEL_FILE_PATH)
    print(f"Excel file '{EXCEL_FILE_PATH}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: '{EXCEL_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    excel_df = pd.DataFrame()

# Load CSV
try:
    csv_df = pd.read_csv(CSV_FILE_PATH)
    print(f"CSV file '{CSV_FILE_PATH}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: '{CSV_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    csv_df = pd.DataFrame()

# Load Verification CSV
try:
    verification_df = pd.read_csv(VERIFICATION_FILE_PATH)
    print(f"Verification file '{VERIFICATION_FILE_PATH}' loaded successfully.")
except FileNotFoundError:
    print(f"Error: '{VERIFICATION_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
    verification_df = pd.DataFrame()

# Load JSON Data and Parse Coordinates
json_image_coords = {} # To store extracted data: {image_name: {full_path: ..., questions: {Q_id: {coords}}}}
print("\n--- Loading and Parsing JSON Data ---")
try:
    with open(JSON_FILE_PATH, 'r') as f:
        json_content = json.load(f)
    print(f"JSON file '{JSON_FILE_PATH}' loaded successfully.")

    if "IMAGES" in json_content:
        for image_entry in json_content["IMAGES"]:
            full_image_path_from_json = image_entry.get("IMAGENAME", "")
            if full_image_path_from_json:
                # Extract just the filename (e.g., BATCH05001.jpg)
                image_name_only = os.path.basename(full_image_path_from_json)
                
                json_image_coords[image_name_only] = {
                    'full_image_path': full_image_path_from_json, # Store the full path from JSON
                    'questions': {}
                }

                for field_entry in image_entry.get("FIELDS", []):
                    field_name = field_entry.get("FIELD", "")
                    if field_name.startswith("question_"):
                        # Convert "question_1" to "Q1"
                        q_number_formatted = field_name.replace("question_", "Q")
                        
                        try:
                            # Convert coordinates to integers
                            x = int(field_entry.get("XCORD"))
                            y = int(field_entry.get("YCORD"))
                            w = int(field_entry.get("WIDTH"))
                            h = int(field_entry.get("HEIGHT"))
                            
                            json_image_coords[image_name_only]['questions'][q_number_formatted] = {
                                'x': x, 'y': y, 'width': w, 'height': h
                            }
                        except (ValueError, TypeError):
                            print(f"Warning: Could not parse coordinates for {image_name_only}, {field_name}. Skipping.")
                            continue
    else:
        print("Warning: 'IMAGES' key not found in JSON data.")

except FileNotFoundError:
    print(f"Error: '{JSON_FILE_PATH}' not found. Please upload it to Colab or provide the correct path.")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from '{JSON_FILE_PATH}'. Check file format.")
except Exception as e:
    print(f"An unexpected error occurred while loading JSON: {e}")

# Create directory for cropped images if it doesn't exist
if not os.path.exists(CROPPED_IMAGES_DIR):
    os.makedirs(CROPPED_IMAGES_DIR)
    print(f"Created directory: {CROPPED_IMAGES_DIR}")

# --- 2. Drop Columns from CSV DataFrame ---

if not csv_df.empty:
    columns_to_drop_csv = ['Roll Number', 'Registration Number', 'Question Booklet Number']
    csv_df = csv_df.drop(columns=columns_to_drop_csv, errors='ignore')
    print("\nDropped specified columns from CSV DataFrame.")
else:
    print("\nCSV DataFrame is empty, skipping column drop.")

# --- 3. Prepare Data for Comparison ---

comparison_results = [] # To store the final structured output
if not excel_df.empty and not csv_df.empty:
    # Standardize the ImageName column for merging
    if 'Image Name' in csv_df.columns and 'ImageName' not in csv_df.columns:
        csv_df = csv_df.rename(columns={'Image Name': 'ImageName'})
        print("Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.")
    # Assuming Excel already has 'ImageName' based on previous context, but good to check.
    if 'Image Name' in excel_df.columns and 'ImageName' not in excel_df.columns:
        excel_df = excel_df.rename(columns={'Image Name': 'ImageName'})
        print("Renamed 'Image Name' in Excel DataFrame to 'ImageName' for consistent merging.")

    # Get the list of question columns (Q1 to Q40) based on the excel_df's columns, excluding 'ImageName'
    question_columns = [col for col in excel_df.columns if col.startswith('Q')]
    if not question_columns:
        print("Warning: No 'Q' columns (Q1-Q40) found in Excel file. Cannot perform question-level comparison.")

    # Merge the dataframes on 'ImageName'
    merged_df = pd.merge(excel_df, csv_df, on='ImageName', how='outer', suffixes=('_excel', '_csv'))

    print("\n--- Performing Comparison and Collecting Differences ---")
    print("Differences will be logged. Cropped images for differing questions will be saved.")

    if not merged_df.empty and question_columns:
        # Prepare verification_df for easier lookup
        verification_df_indexed = pd.DataFrame() # Initialize as empty
        if not verification_df.empty:
            if 'Image Name' in verification_df.columns and 'ImageName' not in verification_df.columns:
                verification_df = verification_df.rename(columns={'Image Name': 'ImageName'})
            
            if 'ImageName' in verification_df.columns:
                verification_df_indexed = verification_df.set_index('ImageName')
                print("Verification DataFrame indexed for faster lookups.")
            else:
                print("Warning: 'ImageName' column not found in verification.csv after potential rename. Cannot use verification data.")
        else:
            print("Verification DataFrame is empty, cannot add option details.")

        for index, row in merged_df.iterrows():
            image_name = row['ImageName']
            
            # Check if ImageName exists in both original DFs
            in_excel = image_name in excel_df['ImageName'].values if not excel_df.empty else False
            in_csv = image_name in csv_df['ImageName'].values if not csv_df.empty else False

            # Handle cases where ImageName is only in one file
            if not in_excel and in_csv:
                # Log this difference with N/A for question number and appropriate notes
                comparison_results.append({
                    'ImageName': image_name,
                    'QuestionNumber': 'N/A',
                    'Excel Result': 'Image not in Excel',
                    'CSV Result': 'Image in CSV',
                    'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan, # Add options as NaN
                    'Cropped Image Link': np.nan # No specific question to crop
                })
                continue # Move to the next ImageName

            elif in_excel and not in_csv:
                comparison_results.append({
                    'ImageName': image_name,
                    'QuestionNumber': 'N/A',
                    'Excel Result': 'Image in Excel',
                    'CSV Result': 'Image not in CSV',
                    'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan, # Add options as NaN
                    'Cropped Image Link': np.nan # No specific question to crop
                })
                continue # Move to the next ImageName

            # Process questions for ImageNames present in both (or only one with specific question diffs)
            for q_col in question_columns:
                excel_value = row.get(f'{q_col}_excel')
                csv_value = row.get(f'{q_col}_csv')

                is_excel_na = pd.isna(excel_value)
                is_csv_na = pd.isna(csv_value)

                # Only log if values are different (including one being NaN and other not)
                if not (is_excel_na and is_csv_na) and \
                   (is_excel_na != is_csv_na or (not is_excel_na and not is_csv_na and excel_value != csv_value)):
                    
                    result_row = {
                        'ImageName': image_name,
                        'QuestionNumber': q_col,
                        'Excel Result': excel_value,
                        'CSV Result': csv_value
                    }

                    # --- Fetching data from verification.csv ---
                    option_results = {'A': np.nan, 'B': np.nan, 'C': np.nan, 'D': np.nan}
                    if not verification_df_indexed.empty and image_name in verification_df_indexed.index:
                        ver_row = verification_df_indexed.loc[image_name]
                        q_num_stripped = q_col.replace('Q', '') # e.g., '1' from 'Q1'

                        for option in ['A', 'B', 'C', 'D']:
                            option_col_name = f"{q_num_stripped}{option}"
                            # result_option_col_name = f"Result {q_num_stripped}{option}"
                            result_option_col_name = f"{q_num_stripped}{option}"
                            
                            if result_option_col_name in ver_row and not pd.isna(ver_row[result_option_col_name]):
                                option_results[option] = ver_row[result_option_col_name]
                            elif option_col_name in ver_row and not pd.isna(ver_row[option_col_name]):
                                option_results[option] = ver_row[option_col_name]
                    result_row.update(option_results)

                    # --- New: Fetching and Cropping Image ---
                    cropped_image_path_for_excel = np.nan # Default to NaN

                    if image_name in json_image_coords and q_col in json_image_coords[image_name]['questions']:
                        img_details = json_image_coords[image_name]
                        q_coords = img_details['questions'][q_col]
                        
                        # Construct the full path to the original image file
                        image_filename_to_open = os.path.join(ORIGINAL_IMAGES_FOLDER_PATH, os.path.basename(img_details['full_image_path']))
                        
                        try:
                            # Open the image using Pillow
                            with Image.open(image_filename_to_open) as img:
                                # Define bounding box (left, upper, right, lower)
                                box = (q_coords['x'], q_coords['y'],
                                       q_coords['x'] + q_coords['width'],
                                       q_coords['y'] + q_coords['height'])
                                
                                cropped_img = img.crop(box)
                                
                                # Define path to save the cropped image
                                cropped_filename = f"{image_name}_{q_col}.png"
                                cropped_save_path = os.path.join(CROPPED_IMAGES_DIR, cropped_filename)
                                
                                cropped_img.save(cropped_save_path)
                                print(f"Cropped and saved: {cropped_save_path}") # Uncomment for verbose output
                                
                                # Store the relative path for Excel to create a hyperlink
                                # For local viewing after downloading, a relative path is often best.
                                # The 'file://' prefix often helps Excel recognize it as a link.
                                cropped_image_path_for_excel = f"file://{os.path.abspath(cropped_save_path)}"


                        except FileNotFoundError:
                            print(f"Warning: Original image '{image_filename_to_open}' not found for cropping '{image_name} {q_col}'. Please ensure it is in '{ORIGINAL_IMAGES_FOLDER_PATH}' folder.")
                        except Exception as e:
                            print(f"Error cropping image '{image_name} {q_col}': {e}")
                    
                    result_row['Cropped Image Link'] = cropped_image_path_for_excel
                    comparison_results.append(result_row)
    else:
        print("Merged DataFrame is empty or no question columns found. No comparison can be made.")

else:
    print("\nCannot perform comparison as one or more required dataframes (Excel/CSV/JSON) are empty. Please check file paths and content.")

# --- 4. Create and Save Output Excel ---

if comparison_results:
    output_df = pd.DataFrame(comparison_results)
    
    # Define the desired order of columns
    output_columns = ['ImageName', 'QuestionNumber', 'Excel Result', 'CSV Result', 'A', 'B', 'C', 'D', 'Cropped Image Link']
    # Reindex the DataFrame to ensure desired column order, filling missing columns with NaN
    output_df = output_df.reindex(columns=output_columns, fill_value=np.nan)

    try:
        output_df.to_excel(OUTPUT_EXCEL_FILE, index=False)
        from openpyxl import load_workbook
        from openpyxl.drawing.image import Image as XLImage

        # Load the Excel file we just created
        wb = load_workbook(OUTPUT_EXCEL_FILE)
        ws = wb.active

        # Find column index of "Cropped Image Link"
        image_col_idx = None
        for idx, col_name in enumerate(ws[1], start=1):
            if col_name.value == "Cropped Image Link":
                image_col_idx = idx
                break

        # Set column width for images
        if image_col_idx:
            col_letter = ws.cell(row=1, column=image_col_idx).column_letter
            ws.column_dimensions[col_letter].width = 25  # make column wide enough

            for row in range(2, ws.max_row + 1):
                img_path = ws.cell(row=row, column=image_col_idx).value
                if img_path and isinstance(img_path, str) and os.path.exists(img_path.replace("file://", "")):
                    try:
                        img_path_clean = img_path.replace("file://", "")
                        img = XLImage(img_path_clean)

                        # --- Resize image (keep aspect ratio) ---
                        base_width = 120
                        w_percent = base_width / float(img.width)
                        h_size = int(float(img.height) * w_percent)
                        img.width = base_width
                        img.height = h_size

                        # Set row height to match image
                        ws.row_dimensions[row].height = h_size * 0.75  # Excel height conversion factor

                        # Insert image in the cell
                        cell_address = f"{col_letter}{row}"
                        ws.add_image(img, cell_address)

                        # Remove text path
                        ws.cell(row=row, column=image_col_idx).value = None

                    except Exception as e:
                        print(f"Error inserting image for row {row}: {e}")

        wb.save(OUTPUT_EXCEL_FILE)
        print(f"Images embedded into '{OUTPUT_EXCEL_FILE}' with corrected aspect ratio.")

    except Exception as e:
        print(f"\nError saving Excel file: {e}")
else:
    print("\nNo differences found or data to save. Output Excel file not created.")

--- Loading Data ---
Excel file 'D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\Files\BATCH05out.xlsx' loaded successfully.
CSV file 'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\options_BATCH05\ed_results_human.csv' loaded successfully.
Verification file 'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\options_BATCH05\verification.csv' loaded successfully.

--- Loading and Parsing JSON Data ---
JSON file 'D:\Projects\OMR\new_abhigyan\Restructure\Images\HSOMR\25072025\Output\BATCH05\BATCH05.json' loaded successfully.

Dropped specified columns from CSV DataFrame.
Renamed 'Image Name' in CSV DataFrame to 'ImageName' for consistent merging.

--- Performing Comparison and Collecting Differences ---
Differences will be logged. Cropped images for differing questions will be saved.
Verification DataFrame indexed for faster lookups.
Cropped and saved: cropped_question_images\BATCH05010.jpg_Q9.png
Cropped and sav

In [ ]:
# import pandas as pd

# # Load the Excel file
# excel1_file = r"D:\Projects\OMR\new_abhigyan\Restructure\Codes_Integration\Delete\comparison_results.xlsx"  # Replace with your file path
# df = pd.read_excel(excel1_file)

# # Drop specified columns
# columns_to_drop = ["ImageName", "CroppedImageLink"]
# df = df.drop(columns=columns_to_drop, errors='ignore')  # 'ignore' avoids error if column doesn't exist

# # Save as CSV
# csv1_file = "output.csv"  # Desired output filename
# df.to_csv(csv1_file, index=False)

# print(f"CSV saved as {csv1_file}")

CSV saved as output.csv


In [ ]:
For Marked:
        A       B       C       D
Marked  51.44   33.83   33.08   74.49   
Partial 185     185     185     185
Blank   230     230     233     228